# Webscrape 1

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import time
from tqdm import tqdm
import pickle as pkl

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

In [20]:
all_models_hrefs = pkl.load(open('webscrape_c/all_models_hrefs_stage2.p','rb'))

In [3]:
# pkl.dump([], open('test.p','wb'))

In [19]:
root_url ='https://www.carspecs.us'

In [12]:
def soups_(url):
    r = requests.get(url)
    print(r.status_code)
    
    return BeautifulSoup(r.content, 'html.parser')

In [13]:
list_of_cars_for_consideration = ['Acura',
'Alfa-Romeo',
'Aston-Martin',
'Audi',
'Bentley',
'BMW',
'Buick',
'Cadillac',
'Chevrolet',
'Chrysler',
'Dodge',
'Ferrari',
'FIAT',
'Ford',
'Genesis',
'GMC',
'Honda',
'Hyundai',
'INFINITI',
'Jaguar',
'Jeep',
'Kia',
'Lamborghini',
'Land-Rover',
'Lexus',
'Lincoln',
'Lotus',
'Maserati',
'MAZDA',
'McLaren',
'Mercedes-Benz',
'MINI',
'Mitsubishi',
'Nissan',
'Porsche',
'Ram',
'Rolls-Royce',
'smart',
'Subaru',
'Tesla',
'Toyota',
'Volkswagen',
'Volvo']
print(len(list_of_cars_for_consideration))

43


In [14]:
brand_list_urls = list(map(lambda x: 'https://www.carspecs.us/cars/2019/'+x.lower(), list_of_cars_for_consideration))

In [15]:
brand_list_urls;

In [16]:
def carspecs_parser_models(urls):
    dict_ = {}
    for url in urls:
        soup1 = soups_(url).findAll('ul')[2]
        soup2 = soups_(url).findAll('div',{'class':"pure-u-1"})[1]
        model_hrefs = list(map(lambda x:x.attrs['href'], soup2.findAll('a')))
        if len(model_hrefs) > 0:
            dict_[url]= model_hrefs
        else:
            url=url.replace('2019','2018')
            soup1 = soups_(url).findAll('ul')[2]
            soup2 = soups_(url).findAll('div',{'class':"pure-u-1"})[1]
            model_hrefs = list(map(lambda x:x.attrs['href'], soup2.findAll('a')))
            if len(model_hrefs) > 0:
                dict_[url]= model_hrefs
            else:
                url=url.replace('2018','2017')
                soup1 = soups_(url).findAll('ul')[2]
                soup2 = soups_(url).findAll('div',{'class':"pure-u-1"})[1]
                model_hrefs = list(map(lambda x:x.attrs['href'], soup2.findAll('a')))
                dict_[url]= model_hrefs
        print(model_hrefs)
    return dict_

In [10]:
# all_models_hrefs = carspecs_parser_models(brand_list_urls)

200
200
['/cars/2019/acura/ilx', '/cars/2019/acura/mdx', '/cars/2019/acura/nsx', '/cars/2019/acura/rdx', '/cars/2019/acura/rlx', '/cars/2019/acura/tlx']
200
200
200
200
['/cars/2018/alfa-romeo/4c', '/cars/2018/alfa-romeo/4c-spider', '/cars/2018/alfa-romeo/giulia', '/cars/2018/alfa-romeo/stelvio']
200
200
['/cars/2019/aston-martin/db11', '/cars/2019/aston-martin/vanquish', '/cars/2019/aston-martin/vantage']
200
200
['/cars/2019/audi/a3', '/cars/2019/audi/a3-cabriolet', '/cars/2019/audi/a3-sedan', '/cars/2019/audi/a4', '/cars/2019/audi/a4-allroad', '/cars/2019/audi/a5', '/cars/2019/audi/a5-sportback', '/cars/2019/audi/a6', '/cars/2019/audi/a7', '/cars/2019/audi/a8', '/cars/2019/audi/a8-l', '/cars/2019/audi/e-tron', '/cars/2019/audi/q3', '/cars/2019/audi/q5', '/cars/2019/audi/q7', '/cars/2019/audi/q8', '/cars/2019/audi/rs-3', '/cars/2019/audi/rs-5-coupe', '/cars/2019/audi/s3', '/cars/2019/audi/s4', '/cars/2019/audi/s5-cabriolet', '/cars/2019/audi/s5-coupe', '/cars/2019/audi/s5-sportback',

200
['/cars/2019/porsche/718', '/cars/2019/porsche/911', '/cars/2019/porsche/cayenne', '/cars/2019/porsche/macan', '/cars/2019/porsche/panamera']
200
200
['/cars/2019/ram/1500', '/cars/2019/ram/1500-classic', '/cars/2019/ram/2500', '/cars/2019/ram/promaster', '/cars/2019/ram/promaster-city']
200
200
['/cars/2019/rolls-royce/cullinan', '/cars/2019/rolls-royce/dawn', '/cars/2019/rolls-royce/ghost', '/cars/2019/rolls-royce/phantom', '/cars/2019/rolls-royce/wraith']
200
200
['/cars/2019/smart/fortwo']
200
200
['/cars/2019/subaru/ascent', '/cars/2019/subaru/brz', '/cars/2019/subaru/crosstrek', '/cars/2019/subaru/forester', '/cars/2019/subaru/legacy', '/cars/2019/subaru/outback', '/cars/2019/subaru/wrx']
200
200
['/cars/2019/tesla/model-3', '/cars/2019/tesla/model-s', '/cars/2019/tesla/model-x']
200
200
['/cars/2019/toyota/4runner', '/cars/2019/toyota/86', '/cars/2019/toyota/avalon', '/cars/2019/toyota/camry', '/cars/2019/toyota/c-hr', '/cars/2019/toyota/corolla', '/cars/2019/toyota/corolla-

In [11]:
# len(all_models_hrefs)

43

In [12]:
# all_models_hrefs;

In [13]:
# pkl.dump(all_models_hrefs, open('webscrape_c/all_models_hrefs_stage2.p','wb'))

## Helper Functions

In [17]:
def latest_model_url(urls):
    empty_list = []
    for k,v in urls.items():
        time.sleep(1)
        for url in v:
            dict_={}
            model_variations_list = soups_(root_url+url).find('ul', {'class':'car-trim-list'})
            print(root_url+url)
            model_variations_urls = list(map(lambda x:x.attrs['href'] ,model_variations_list.findAll('a')))
            dict_[k]=model_variations_urls
            empty_list.append(dict_)
            time.sleep(1)
                                         
    return empty_list

In [ ]:
# pkl.dump([], open('test.p','wb'))

In [ ]:
# type_bs4 = type(soup1[0])

In [ ]:
# for item in all_model_list_2:
#     for k,v in list(item.items()):
#         for k1,v1 in list(v.items()):
#             print(type(k1))
#             if type(k1) == type_bs4:
#                 v['extra_options'] = k1.text
#                 del v[k1]

## All Models Variations

In [10]:
all_model_variations_urls = pkl.load(open('webscrape_c/all_model_variations_urls.p','rb'))

In [16]:
# all_model_variations_urls = latest_model_url(all_models_hrefs)

In [18]:
# pkl.dump(all_model_variations_urls, open('webscrape_c/all_model_variations_urls.p','wb'))

In [308]:
all_model_variations_urls.index({'https://www.carspecs.us/cars/2019/audi': ['/cars/2019/audi/a3-cabriolet/77677',
   '/cars/2019/audi/a3-cabriolet/77678',
   '/cars/2019/audi/a3-cabriolet/77679',
   '/cars/2019/audi/a3-cabriolet/77680',
   '/cars/2019/audi/a3-cabriolet/77681',
   '/cars/2019/audi/a3-cabriolet/77682']})

14

In [19]:
all_model_variations_urls[14];

In [360]:
all_model_variations_urls.index({'https://www.carspecs.us/cars/2019/audi': ['/cars/2019/audi/e-tron/79118',
   '/cars/2019/audi/e-tron/79119']},)

24

In [28]:
all_model_variations_urls[173:]

[{'https://www.carspecs.us/cars/2017/infiniti': ['/cars/2017/infiniti/q70/68694',
   '/cars/2017/infiniti/q70/68695',
   '/cars/2017/infiniti/q70/46795',
   '/cars/2017/infiniti/q70/46796',
   '/cars/2017/infiniti/q70/46797',
   '/cars/2017/infiniti/q70/46798',
   '/cars/2017/infiniti/q70/46803',
   '/cars/2017/infiniti/q70/46799',
   '/cars/2017/infiniti/q70/46800',
   '/cars/2017/infiniti/q70/46801',
   '/cars/2017/infiniti/q70/46802']},
 {'https://www.carspecs.us/cars/2017/infiniti': ['/cars/2017/infiniti/qx30/46804',
   '/cars/2017/infiniti/qx30/68696',
   '/cars/2017/infiniti/qx30/46805',
   '/cars/2017/infiniti/qx30/46806',
   '/cars/2017/infiniti/qx30/46807',
   '/cars/2017/infiniti/qx30/46808',
   '/cars/2017/infiniti/qx30/46809']},
 {'https://www.carspecs.us/cars/2017/infiniti': ['/cars/2017/infiniti/qx50/46810',
   '/cars/2017/infiniti/qx50/46811',
   '/cars/2017/infiniti/qx50/68697']},
 {'https://www.carspecs.us/cars/2017/infiniti': ['/cars/2017/infiniti/qx60/74663',
   '/ca

In [29]:
all_model_variations_urls.index({'https://www.carspecs.us/cars/2018/lotus': ['/cars/2018/lotus/evora/77579']})

228

## Get the specs for all

In [7]:
def spec_parser(urls, pkl_filename_path):
    model_list = []
    model_list = pkl.load(open(pkl_filename_path,'rb'))
    print(model_list)
    for url in urls:
        for k, model_urls in url.items():
            model_dict = {}
            print(model_dict)
            for model_url in model_urls:
                spec_dict={}
                print(spec_dict)
                soup_list1 = soups_(root_url+model_url).find('div',{"class":"main-car-details"}).text.strip()
                if re.findall('\$.*[\d]',soup_list1):
                    spec_dict['price'] = int(re.findall('\$.*[\d]',soup_list1)[0].replace(',','').replace('$',''))
                else:
                    spec_dict['price'] = 'price not given'
                soup_list2 = soups_(root_url+model_url).findAll('div',{"class":"pure-u-1 pure-u-md-1-2"})
                for spec in soup_list2:
                    time.sleep(1)
                    try:
                        if len(spec.text.split('\r\n')) >= 2:
                            components = spec.text.split('\r\n')
                            spec_dict[components[0].strip()]=components[1].strip()
                        else:
                            spec_dict["extra options"] = spec.text
                    except:
                        print('something went wrong')
                        continue
                model_dict[model_url] = spec_dict
                print('Success', ':', model_dict)
            model_list = pkl.load(open(pkl_filename_path,'rb'))
            model_list.append(model_dict)
            print('Success', ':', model_list)
            pkl.dump(model_list, open(pkl_filename_path,'wb'))               
    return model_list

In [8]:
import sys

In [9]:
sys.setrecursionlimit(10000)

#### Batch 2

In [25]:
# model_list_2 = pkl.load(open('webscrape_c/model_list_2.p','rb'))

In [358]:
# all_model_list_2 = []
# all_model_list_2 = pkl.dump([], open('webscrape_c/model_list_2.p','wb'))

In [11]:
# all_model_list_2 = spec_parser(all_model_variations_urls[14:24], 'webscrape_c/model_list_2.p')

In [513]:
#pkl.dump(all_model_list_2, open('webscrape_c/model_list_2.p','wb'))

#### Batch 3

In [361]:
# all_model_list_3 = []
# all_model_list_3 = pkl.dump([], open('webscrape_c/model_list_3.p','wb'))

In [10]:
# all_model_list_3 = spec_parser(all_model_variations_urls[25:50], 'webscrape_c/model_list_3.p')

In [516]:
# type(all_model_list_3);

list

In [517]:
# pkl.dump(all_model_list_3, open('webscrape_c/model_list_3.p','wb'))

In [9]:
# model_list_3 = pkl.load(open('webscrape_c/model_list_3.p','rb'))

#### Batch 4

In [363]:
# all_model_list_4 = []
# all_model_list_4 = pkl.dump([], open('webscrape_c/model_list_4.p','wb'))

In [8]:
# all_model_list_4 = spec_parser(all_model_variations_urls[50:100], 'webscrape_c/model_list_4.p')

In [525]:
# pkl.dump(all_model_list_4, open('webscrape_c/model_list_4.p','wb'))

In [545]:
# all_model_list_4 = pkl.load(open('webscrape_c/model_list_4.p','rb'))

In [7]:
# all_model_list_4;

In [525]:
# pkl.dump(all_model_list_4, open('webscrape_c/model_list_4.p','wb'))

#### Batch 5 & 6

In [586]:
# all_model_list_5 = []
# all_model_list_5 = pkl.dump([], open('webscrape_c/model_list_5.p','wb'))
# all_model_list_6 = []
# all_model_list_6 = pkl.dump([], open('webscrape_c/model_list_6.p','wb'))

In [5]:
# all_model_list_5 = spec_parser(all_model_variations_urls[100:150], 'webscrape_c/model_list_5.p')

In [4]:
# all_model_list_6 = spec_parser(all_model_variations_urls[150:174], 'webscrape_c/model_list_6.p')

#### Batch 7

In [611]:
# all_model_list_7 = []
# all_model_list_7 = pkl.dump([], open('webscrape_c/model_list_7.p','wb'))

In [6]:
# all_model_list_7 = spec_parser(all_model_variations_urls[174:250], 'webscrape_c/model_list_7.p')

In [ ]:
# all_model_list_7

In [613]:
# pkl.dump(all_model_list_7, open('webscrape_c/model_list_7_back-up.p','wb'))

#### Batch 8

In [3]:
# all_model_list_8 = spec_parser(all_model_variations_urls[229:270], 'webscrape_c/model_list_8.p')

#### Batch 9

In [31]:
all_model_list_9 = []
all_model_list_9 = pkl.dump([], open('webscrape_c/model_list_9.p','wb'))

In [2]:
# all_model_list_9 = spec_parser(all_model_variations_urls[270:300], 'webscrape_c/model_list_9.p')

#### Batch 10

In [5]:
all_model_list_10 = []
all_model_list_10 = pkl.dump([], open('webscrape_c/model_list_10.p','wb'))

In [1]:
# all_model_list_10 = spec_parser(all_model_variations_urls[300:], 'webscrape_c/model_list_10.p')

# Creating DataFrame

In [526]:
pkl_1 = list(map(lambda df: pd.DataFrame(df).T, pkl.load(open('webscrape_c/model_list.p','rb')
pkl_2 =list(map(lambda df: pd.DataFrame(df).T, pkl.load(open('webscrape_c/model_list_2.p','rb'))))
pkl_3 =list(map(lambda df: pd.DataFrame(df).T, pkl.load(open('webscrape_c/model_list_3.p','rb'))))
pkl_4 =list(map(lambda df: pd.DataFrame(df).T, pkl.load(open('webscrape_c/model_list_4.p','rb'))))

In [608]:
pkl_5 =list(map(lambda df: pd.DataFrame(df).T, pkl.load(open('webscrape_c/model_list_5.p','rb'))))
pkl_6 =list(map(lambda df: pd.DataFrame(df).T, pkl.load(open('webscrape_c/model_list_6.p','rb'))))

In [614]:

pkl_7 =list(map(lambda df: pd.DataFrame(df).T, pkl.load(open('webscrape_c/model_list_7.p','rb'))))

In [578]:
df_1 = pd.concat(pkl_1)
df_2 = pd.concat(pkl_2)
df_3 = pd.concat(pkl_3)
df_4 = pd.concat(pkl_4)
df_full = pd.concat([df_1, df_2, df_3, df_4])

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 's

In [609]:
df_5 = pd.concat(pkl_5)
df_6 = pd.concat(pkl_6)
df_full = pd.concat([df_full, df_5, df_6])

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 's

In [615]:
df_7 = pd.concat(pkl_7)
df_full = pd.concat([df_full, df_7])

/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
/Users/flatironschool/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [617]:
df_full

,0-60 mph,3rd Row Head Room,3rd Row Hip Room,3rd Row Leg Room,3rd Row Shoulder Room,4th Row Head Room,4th Row Hip Room,4th Row Leg Room,4th Row Shoulder Room,ABS Brakes Type,...,Transmission,Turning circle,Valves,Weight Distributing Hitch - Max Tongue Weight,Weight Distributing Hitch - Max Trailer Weight,Wheelbase,Width,extra options,extra_options,price
/cars/2019/acura/ilx/79088,6.63 sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Wheel,...,Transmission: 8-Speed Dual-Clutch -inc: torque...,36.80 ft.,NaN,NaN,NaN,105.10 in.,70.60,NaN,NaN,25900
/cars/2019/acura/ilx/79090,6.68 sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Wheel,...,Transmission: 8-Speed Dual-Clutch -inc: torque...,36.80 ft.,NaN,NaN,NaN,105.10 in.,70.60,NaN,NaN,27650
/cars/2019/acura/ilx/79089,6.71 sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Wheel,...,Transmission: 8-Speed Dual-Clutch -inc: torque...,36.80 ft.,NaN,NaN,NaN,105.10 in.,70.60,NaN,NaN,29650
/cars/2019/acura/ilx/79092,6.68 sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Wheel,...,Transmission: 8-Speed Dual-Clutch -inc: torque...,36.80 ft.,NaN,NaN,NaN,105.10 in.,70.60,NaN,NaN,29550
/cars/2019/acura/ilx/79091,6.72 sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-Wheel,...,Transmission: 8-Speed Dual-Clutch -inc: torque...,36.80 ft.,NaN,NaN,NaN,105.10 in.,70.60,NaN,NaN,31550
/cars/2019/acura/mdx/77624,7.00 sec,35.60 in.,40.60 in.,28.10 in.,54.70 in.,NaN,NaN,NaN,NaN,4-Wheel,...,Transmission: 9-Speed Automatic -inc: Sequenti...,38.70 ft.,NaN,350 lbs,"3,500 lbs",111.00 in.,77.70,NaN,NaN,44300
/cars/2019/acura/mdx/77621,7.07 sec,35.60 in.,40.60 in.,28.10 in.,54.70 in.,NaN,NaN,NaN,NaN,4-Wheel,...,Transmission: 9-Speed Automatic -inc: Sequenti...,38.70 ft.,NaN,350 lbs,"3,500 lbs",111.00 in.,77.70,NaN,NaN,56050
/cars/2019/acura/mdx/77620,7.13 sec,35.60 in.,40.60 in.,28.10 in.,54.70 in.,NaN,NaN,NaN,NaN,4-Wheel,...,Transmission: 9-Speed Automatic -inc: Sequenti...,38.70 ft.,NaN,350 lbs,"3,500 lbs",111.00 in.,77.70,NaN,NaN,58050
/cars/2019/acura/mdx/77623,7.03 sec,35.60 in.,40.60 in.,28.10 in.,54.70 in.,NaN,NaN,NaN,NaN,4-Wheel,...,Transmission: 9-Speed Automatic -inc: Sequenti...,38.70 ft.,NaN,350 lbs,"3,500 lbs",111.00 in.,77.70,NaN,NaN,49300
/cars/2019/acura/mdx/77622,7.03 sec,35.60 in.,40.60 in.,28.10 in.,54.70 in.,NaN,NaN,NaN,NaN,4-Wheel,...,Transmission: 9-Speed Automatic -inc: Sequenti...,38.70 ft.,NaN,350 lbs,"3,500 lbs",111.00 in.,77.70,NaN,NaN,51300
